In [1]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
from source.tserie import TSerie
from source.read_ontario import read_ontario, read_ontario_stations
from source.read_hongkong import read_hongkong
from source.read_brasil import read_brasil
from source.app_dataset import OntarioDataset, BrasilDataset, HongKongDataset
from source.utils import commonWindows
from source.torch_utils import getContrastiveFeatures
import aqi


/home/texs/anaconda3/envs/clearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Months case

In [5]:
dataset = OntarioDataset(fill_missing=True, granularity='years')
# dataset = OntarioDataset(fill_missing=False, granularity='years')

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [3]:
from source.utils import AVAILABLE_POLUTANTS, daily_iaqi
import aqi

AVAILABLE_POLUTANTS
def format_date(date, gran):
    if gran == 'daily':
        return '{}_{}_{}'.format( date.year, date.month, date.day)
    elif gran == 'monthly':
        return '{}_{}'.format( date.year, date.month)
    elif gran == 'annualy':
        return '{}'.format(date.year)

def get_annualy_iaqis_map(daily_iaqi):
    iaqi = {}
    for polK, polD in daily_iaqi.items():
        if  polK not in AVAILABLE_POLUTANTS:
            continue
        iaqi[polK] = {}
        for statK, statD in polD.items():
            iaqi[polK][statK] = {}
            for yearK, yearD in statD.items():
                values = []
                for monthK, monthD in yearD.items():
                    values = values + list(monthD.values())
                    
                values = np.array(values)
                iaqi[polK][statK][yearK] = np.mean(values)
                
    return iaqi

def get_monthly_iaqis_map(daily_iaqi):
    monthly_iaqi = {}
    for polK, polD in daily_iaqi.items():
        if  polK not in AVAILABLE_POLUTANTS:
            continue
        monthly_iaqi[polK] = {}
        for statK, statD in polD.items():
            monthly_iaqi[polK][statK] = {}
            for yearK, yearD in statD.items():
                monthly_iaqi[polK][statK][yearK] = {}
                for monthK, monthD in yearD.items():
                    # print(np.array(list(monthD.values())))
                    values = np.array(list(monthD.values()))
                    monthly_iaqi[polK][statK][yearK][monthK] = np.mean(values)
                
    return monthly_iaqi

def get_daily_iaqis_map(win_map):
    windows_aqis = {}
    for polK, polD in win_map.items():
        if  polK in AVAILABLE_POLUTANTS:
            windows_aqis[polK] = {}
            for statK, statD in polD.items():
                windows_aqis[polK][statK] = {}
                for dateK, dateD in statD.items():
                    date = dateD[1]
                    data = dateD[0]
                    if dateD[0].shape[0] != 24:
                        print('Error, not 24 days')
                    else:
                        try:
                            if date.year not in windows_aqis[polK][statK]:
                                windows_aqis[polK][statK][date.year] = {}
                                
                            if date.month not in windows_aqis[polK][statK][date.year]:
                                windows_aqis[polK][statK][date.year][date.month] = {}

                            windows_aqis[polK][statK][date.year][date.month][date.day] = int(daily_iaqi(polK, data))
                        except:
                            print('Exception while getting daily_iaqi')
    return windows_aqis

daily_iaqis = get_daily_iaqis_map(dataset.windows_map)
monthly_iaqis = get_monthly_iaqis_map(daily_iaqis)
annualy_iaqis = get_annualy_iaqis_map(daily_iaqis)


np.save("{}_daily_iaqis.npy".format(dataset.name), daily_iaqis)

In [8]:
# dataset = HongKongDataset(fill_missing=True, granularity='years')
dataset = HongKongDataset(fill_missing=True, granularity='daily')
# dataset = BrasilDataset(fill_missing=True, granularity='daily')
# dataset = OntarioDataset(fill_missing=True, granularity='daily')
# dataset_daily = OntarioDataset(fill_missing=True, granularity='years')
# dataset = OntarioDataset(fill_missing=True, granularity='years')

In [3]:
# print(dataset.windows_original_map['NO2']['Brampton']['2000-07-30 00:00:00'])
# print(dataset.windows_map['NO2']['Brampton']['2000-07-30 00:00:00'])



In [7]:
# from source.utils import getCommonDates


# # station = 'TSEUNG KWAN O'
# # station = 'SOUTHERN'
# # station = 'KWAI CHUNG'
# # list(dataset.windows_map['RSP']['CENTRAL'].keys())[-100:]
# # dataset.common_windows(['NO', 'NOx', 'NO2', 'SO2', 'CO', 'O3', 'PM25'], ['CENTRAL'])




# # getCommonDates(dataset.windows_map, 'CENTRAL', ['NO', 'NOx', 'NO2', 'SO2', 'CO', 'O3', 'PM25'])
# # getCommonDates(dataset.windows_map, 'Santana', ['MP25'])
# dataset.windows_map['CO']['Pinheiros'].keys()


dict_keys([])

In [3]:
# from source.utils import getCommonDates
dataset.window_stations[dataset.window_station_ids]
dataset.window_stations
# dataset_daily.windows_map['CO']['KWAI CHUNG'].keys()


def intersection(lst1, lst2): 
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3 

def getCommonDates(windows_map, station, pollutants):
    common_dates = []
    first = True
    
    for pol in pollutants:
        stationDates = list(windows_map[pol][station].keys())
        print(pol)
        print(stationDates)
        if len(common_dates) == 0 and first:
            common_dates = stationDates
        else:
            common_dates = intersection(common_dates, stationDates)
        first = False
    return common_dates


getCommonDates(dataset.windows_map, 'CENTRAL', ['NO', 'NOx', 'NO2', 'SO2', 'CO', 'O3', 'PM25'])
# dataset.window_iaqis

# dataset.windows_map['CO']['Hamilton Downtown'].keys()
# dataset_daily.windows_map['CO']['Hamilton Downtown'].keys()
# daily_iaqis['CO']['Hamilton Downtown'].keys()
# getCommonDates(dataset_daily.windows_map, 'Hamilton Downtown', ['NO2', 'NOX', 'O3', 'SO2', 'RSP', 'FSP', 'CO'])w
# getCommonDates(dataset.windows_map, 'TAP MUN', ['NO2', 'NOX', 'O3', 'SO2', 'RSP', 'FSP', 'CO'])

AttributeError: 'HongKongDataset' object has no attribute 'window_stations'

['O3', 'PM25', 'PM10', 'FSP', 'NO2', 'SO2', 'CO', 'MP25', 'MP10', 'FSP', 'RSP']

In [9]:
from source.utils import AVAILABLE_POLUTANTS, daily_iaqi
import aqi

AVAILABLE_POLUTANTS
def format_date(date, gran):
    if gran == 'daily':
        return '{}_{}_{}'.format( date.year, date.month, date.day)
    elif gran == 'monthly':
        return '{}_{}'.format( date.year, date.month)
    elif gran == 'annualy':
        return '{}'.format(date.year)

def get_annualy_iaqis_map(daily_iaqi):
    iaqi = {}
    for polK, polD in daily_iaqi.items():
        if  polK not in AVAILABLE_POLUTANTS:
            continue
        iaqi[polK] = {}
        for statK, statD in polD.items():
            iaqi[polK][statK] = {}
            for yearK, yearD in statD.items():
                values = []
                for monthK, monthD in yearD.items():
                    values = values + list(monthD.values())
                    
                values = np.array(values)
                iaqi[polK][statK][yearK] = np.mean(values)
                
    return iaqi

def get_monthly_iaqis_map(daily_iaqi):
    monthly_iaqi = {}
    for polK, polD in daily_iaqi.items():
        if  polK not in AVAILABLE_POLUTANTS:
            continue
        monthly_iaqi[polK] = {}
        for statK, statD in polD.items():
            monthly_iaqi[polK][statK] = {}
            for yearK, yearD in statD.items():
                monthly_iaqi[polK][statK][yearK] = {}
                for monthK, monthD in yearD.items():
                    # print(np.array(list(monthD.values())))
                    values = np.array(list(monthD.values()))
                    monthly_iaqi[polK][statK][yearK][monthK] = np.mean(values)
                
    return monthly_iaqi

def get_daily_iaqis_map(win_map):
    windows_aqis = {}
    for polK, polD in win_map.items():
        if  polK in AVAILABLE_POLUTANTS:
            windows_aqis[polK] = {}
            for statK, statD in polD.items():
                windows_aqis[polK][statK] = {}
                for dateK, dateD in statD.items():
                    date = dateD[1]
                    data = dateD[0]
                    if dateD[0].shape[0] != 24:
                        print('Error, not 24 days')
                    else:
                        try:
                            if date.year not in windows_aqis[polK][statK]:
                                windows_aqis[polK][statK][date.year] = {}
                                
                            if date.month not in windows_aqis[polK][statK][date.year]:
                                windows_aqis[polK][statK][date.year][date.month] = {}

                            windows_aqis[polK][statK][date.year][date.month][date.day] = int(daily_iaqi(polK, data))
                        except:
                            print('Exception while getting daily_iaqi')
    return windows_aqis

daily_iaqis = get_daily_iaqis_map(dataset.windows_map)
monthly_iaqis = get_monthly_iaqis_map(daily_iaqis)
annualy_iaqis = get_annualy_iaqis_map(daily_iaqis)


np.save("{}_daily_iaqis.npy".format(dataset.name), daily_iaqis)

Exception while getting daily_iaqi
Exception while getting daily_iaqi
Exception while getting daily_iaqi
Exception while getting daily_iaqi


In [50]:
from source.utils import getCommonDates


pol = 'CO'
# station = 'TSEUNG KWAN O'
# station = 'SOUTHERN'
station = 'Hamilton Downtown'
# print(dataset.windows_map.keys())
# print(dataset.windows_map[pol].keys())
# print(dataset.windows_map[pol][station].keys())
# print(len(dataset.windows_map[pol][station].keys()))
# print(np.unique(list()))
# print(dataset.windows_map[pol][station].keys())

print(dataset.windows_map[pol][station].keys())
# len(getCommonDates(dataset.windows_map, station, ['NO2', 'NOX', 'O3', 'SO2', 'RSP', 'FSP', 'CO']))
# print(daily_iaqis[pol][station].keys())
# print(monthly_iaqis[pol][station].keys())
print(annualy_iaqis[pol][station].keys())
print(daily_iaqis[pol][station].keys())

# annualy_iaqis['FSP']['CENTRAL/WESTERN'].keys()
# daily_iaqis.keys()
# dataset.windows_map['FSP']['CENTRAL/WESTERN'].keys()

dict_keys(['2000-01-01 00:00:00', '2000-01-02 00:00:00', '2000-01-03 00:00:00', '2000-01-04 00:00:00', '2000-01-05 00:00:00', '2000-01-06 00:00:00', '2000-01-07 00:00:00', '2000-01-08 00:00:00', '2000-01-09 00:00:00', '2000-01-10 00:00:00', '2000-01-11 00:00:00', '2000-01-12 00:00:00', '2000-01-13 00:00:00', '2000-01-14 00:00:00', '2000-01-15 00:00:00', '2000-01-16 00:00:00', '2000-01-17 00:00:00', '2000-01-18 00:00:00', '2000-01-19 00:00:00', '2000-01-20 00:00:00', '2000-01-21 00:00:00', '2000-01-22 00:00:00', '2000-01-23 00:00:00', '2000-01-24 00:00:00', '2000-01-25 00:00:00', '2000-01-26 00:00:00', '2000-01-27 00:00:00', '2000-01-28 00:00:00', '2000-01-29 00:00:00', '2000-01-30 00:00:00', '2000-01-31 00:00:00', '2000-02-02 00:00:00', '2000-02-03 00:00:00', '2000-02-04 00:00:00', '2000-02-05 00:00:00', '2000-02-06 00:00:00', '2000-02-07 00:00:00', '2000-02-08 00:00:00', '2000-02-09 00:00:00', '2000-02-10 00:00:00', '2000-02-11 00:00:00', '2000-02-12 00:00:00', '2000-02-13 00:00:00', 

In [6]:
# # np.array(windows_map['CO']['CENTRAL']['1999-01-01 00:00:00'][0]).shape
# # windows_map['SO2']['CAUSEWAY BAY']
# # np.array(windows_map['SO2']['CAUSEWAY BAY']['2022-01-12 00:00:00'][0])
#     # ['1999-01-01 00:00:00'][0])

# # dataset.windows_map['CO']['Chatham']['2006-04-27 00:00:00']
# dataset.windows_map['FSP']['CAUSEWAY BAY']['2023-01-01 00:00:00']

In [7]:


# windows_map = read_hongkong(granularity='daily', cache=True)
# windows_map = read_ontario(granularity='daily', cache=True)
# # windows_map = read_brasil(granularity='daily', cache=True)
# # windows_map = read_hongkong(granularity='months', cache=True)
pollutants = ['FSP']
# # data_info = read_ontario_stations()
# pollutans = POLLUTANTS
# windows, dates, station_ids, stations = commonWindows(windows_map, pollutants, dataset.stations)

# windows, dates, station_ids, stations = ontario.commonWindows(windows_map, pollutants, dataset.stations)
dataset.common_windows(pollutants, dataset.stations)
# print(windows.shape)

KeyError: 'FSP'

In [ ]:
print(dataset.window_iaqis[pollutants[0]].shape)
print(dataset.windows.shape)

(226,)
(226, 365, 1)


In [ ]:
# windows_map['O3'][44008]['2000-01-31 00:00:00'][1].shape
n_map = {}
for pol_k, pol_d in windows_map.items():
    # n_map[pol_k] = {}
    # Eg for 44008, _ in ...
    pol_dict = windows_map[pol_k]
    n_pol_dict = {}
    for sta_k, sta_d in pol_dict.items():
        real_name = data_info[str(sta_k)]['name']
        stat_dict = pol_dict[sta_k]
        
        if real_name in n_pol_dict:
            # print('-')
            real_dict = n_pol_dict[real_name]
            for win_k, win_d in stat_dict.items():
                real_dict[win_k] = win_d
        else:
            n_pol_dict[real_name] = stat_dict
    n_map[pol_k] = n_pol_dict
# data_info

NameError: name 'windows_map' is not defined

In [ ]:
print(n_map.keys())
print(n_map['O3'].keys())
print(len(n_map['O3'].keys()))
print(len(n_map['O3']['Brampton'].keys()))
print(n_map['O3']['Brampton'].keys())

In [ ]:
print(windows_map.keys())
print(windows_map['O3'].keys())
print(len(windows_map['O3'].keys()))
print(len(windows_map['O3'][46089].keys()))
print(windows_map['O3'][46089].keys())

In [ ]:
data_info['46089']

In [ ]:
# windows_map['SO2'][13001]['2010-1'][0].shape
# windows_map
# dataset.stations
windows, dates, station_ids, stations = commonWindows(windows_map, pollutans, dataset.stations)
# windows, dates, station_ids, stations = commonWindows(windows_map, pollutans,['CENTRAL'])


In [ ]:
from skfda.representation.basis import BSpline
from skfda.representation import FDataGrid, FDataBasis

print(windows.shape)


basis = BSpline(n_basis=15, order = 3)

# X = windows[:100, :, 0]
X = windows[:, :, 0]



In [ ]:

import skfda
from skfda.representation.interpolation import SplineInterpolation

import skfda.representation.basis as basis

# dataset = skfda.datasets.fetch_growth()

In [ ]:
# dataset['data']

t = np.arange(X.shape[1])
fd = skfda.FDataGrid(X, t)

fd.data_matrix.shape

In [ ]:
import matplotlib.pyplot as plt

# X.shape
plt.plot(X.transpose())

In [ ]:
# fd_basis = fd.to_basis(basis.BSplineBasis(n_basis=30))
fd_basis = fd.to_basis(basis.BSplineBasis(n_basis=30))
# fd_basis = fd.to_basis(basis.FourierBasis(n_basis=10))

fd_basis.plot()

In [ ]:
from skfda.exploratory.depth import IntegratedDepth
from skfda.exploratory.depth.multivariate import SimplicialDepth, Outlyingness
from skfda.exploratory.visualization import MagnitudeShapePlot

msplot = MagnitudeShapePlot(
    fd,
    multivariate_depth=SimplicialDepth(),
    # multivariate_depth=Outlyingness(),
)
color = 0.3
outliercol = 0.7

msplot.color = color
msplot.outliercol = outliercol
msplot.plot()

In [ ]:
msplot.points.shape

In [ ]:
fd_basis.plot(
    group=msplot.outliers.astype(int),
    group_colors=msplot.colormap([color, outliercol]),
    group_names=['nonoutliers', 'outliers'],
)


In [ ]:
import matplotlib.pyplot as plt


# fd_basis.to_grid().data_matrix

fd_basis = fd.to_basis(basis.BSplineBasis(n_basis=10))
data = fd_basis.to_grid().data_matrix.squeeze()
# data = data[:10].transpose()
data = data.transpose()
plt.plot(data)

# skfda.FDataGrid(fd_basis)
# fd_basis.plot()
# print(type(fd_basis))

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

def fdaOutlier(X):
    
    dts = np.transpose(X)
    print(dts.min())
    print(dts.max())
    print(dts.mean())
    median_vec = np.median(dts, axis=1)
    print('median vec: {}'.format(median_vec))
    mad_vec = stats.median_abs_deviation(dts, axis=1)
    dir_out_matrix = np.subtract(dts.transpose(), median_vec) / mad_vec
    print(dir_out_matrix.min())
    print(dir_out_matrix.max())
    print(dir_out_matrix.mean())
    mean_dir_out = np.mean(dir_out_matrix, axis=1)
    print('-')
    print(mean_dir_out.min())
    print(mean_dir_out.max())
    print(mean_dir_out.mean())
    var_dir_out = np.var(dir_out_matrix, axis=1)
    print('-')
    print(var_dir_out.min())
    print(var_dir_out.max())
    print(var_dir_out.mean())
    return mean_dir_out, var_dir_out

mo, vo = fdaOutlier(X)



plt.scatter(vo, mo)

In [ ]:

# print(pollutans)

filtered_pollutans, filtered_windows = filter_iaqi(pollutans, windows)

print(filtered_windows.shape)
print(filtered_windows.max())
print(filtered_windows.min())
print(filtered_pollutans)
# print(filtered_windows)
# print(filtered_windows.shape)
# print(windows.shape)
aqis = [int(get_aqi(filtered_pollutans, filtered_windows[i])) for i in range(len(filtered_windows))]
print(aqis)

print('iaqis')
iaqis = [int(daily_iaqi(filtered_pollutans[0], filtered_windows[i,:,0])) for i in range(len(filtered_windows))]
print(iaqis)
iaqis = [int(daily_iaqi(filtered_pollutans[1], filtered_windows[i,:,1])) for i in range(len(filtered_windows))]
print(iaqis)
iaqis = [int(daily_iaqi(filtered_pollutans[2], filtered_windows[i,:,2])) for i in range(len(filtered_windows))]
print(iaqis)


In [ ]:
month_ids = np.array([date.month for date in dates])
year_ids = np.array([date.year for date in dates])


stations_map = {int(i):str(stations[i]) for i in range(len(stations))}
months_map = {int(i):str(i) for i in range(1, 13)}
years_map = {int(year) : str(year) for year in year_ids}
# coords = np.array([
#     [
#         float(data_info[str(stations[station_ids[i]])]['latitude']), 
#         float(data_info[str(stations[station_ids[i]])]['longitude'])
#     ]
#     for i in range(len(windows))
# ])

In [ ]:
# dates
import datetime

epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

# unix_time_millis(dates[0])

timeInMs = np.array([unix_time_millis(d) for d in dates])
timeInMs = np.expand_dims(timeInMs, axis=1)


In [ ]:
mts = TSerie(X=windows, y = station_ids)

if SMOOTH > 0:
    mts.smooth(window_size=SMOOTH)

if MODE == 'shape':
    mts.shapeNormalizization()
else:
    mts.minMaxNormalizization()


In [ ]:
from source.featlearn.autoencoder_lr import AutoencoderFL, DCEC, VAE_FL
from sklearn.model_selection import train_test_split

# cae = AutoencoderFL(mts.D, mts.T, feature_size=10)
# cae = DCEC(mts.D, mts.T, feature_size=10, n_clusters=5)
# cae = VAE_FL(mts.D, mts.T, feature_size=10)

X_train, X_val = train_test_split(mts.X.transpose([0, 2, 1]))


# model = BarlowTwins(mts.D, mts.T, feature_size=10, aug_type='noise', encoding_size=6, use_frequency=False)
model = BYOL(mts.D, mts.T, feature_size=7, aug_type='noise', encoding_size=6, use_frequency=False)

# model.fit(mts.X, epochs=1400, batch_size=320)
# model.fit(X_train, epochs=200, batch_size=320, X_val=X_val)
model.fit(X_train, epochs=10, batch_size=320, X_val=X_val)
# cae.fit(mts.X, epochs=1000, batch_size=320, gamma=0)

In [ ]:
from source.featlearn.autoencoder_lr import AutoencoderFL, DCEC




# cae.fit(mts.X, epochs=1000, batch_size=320, gamma=100)
# rec, mts.features, clusters = cae.encode(mts.X)
# rec, mts.features = cae.encode(mts.X)


mts.features = model.encode(mts.X.transpose([0, 2, 1]))


In [ ]:
# mts.features.min()

In [ ]:
# def jitter(x, sigma=0.01):
#     # https://arxiv.org/pdf/1706.00527.pdf
#     return x + np.random.normal(loc=0., scale=sigma, size=x.shape)

# time = X_train[0]
# time = jitter(time)
# plt.plot(time[0])

In [ ]:
from sklearn.metrics import pairwise_distances

delta = 0.
beta = 0.

space_DM = pairwise_distances(coords)
space_DM = space_DM / np.max(space_DM)

feature_DM = pairwise_distances(mts.features, metric='euclidean')
feature_DM = feature_DM / np.max(feature_DM)


time_DM = pairwise_distances(timeInMs)
time_DM = time_DM / np.max(time_DM)

# distM = feature_DM * (1 - delta) + space_DM * delta
distM = feature_DM * (1 - (delta + beta)) + space_DM * delta + time_DM * beta
# mts.features = np.concatenate([mts.features, coords], axis=1)

In [ ]:
# # # clusters
# preds = np.argmax(clusters, axis=1)
# print(np.unique(preds, return_counts=True))

In [ ]:
import umap
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
reducer = umap.UMAP(n_components=2, metric='euclidean')
# reducer = umap.UMAP(n_components=2, metric='precomputed')
# reducer = TSNE(n_components=2, metric='precomputed')

# reducer = UMAP(n_components=2, n_neighbors=10, min_dist=0.1, metric='cosine')
# reducer = UMAP(n_components=2, n_neighbors=10, min_dist=0.1)
# reducer = TSNE(n_components=2, metric='euclidean')

# coords = reducer.fit_transform(mts.features, y = month_ids)
coords = reducer.fit_transform(mts.features)
# coords = reducer.fit_transform(distM)

# plt.scatter(coords[:, 0], coords[:, 1], c=preds)
plt.scatter(coords[:, 0], coords[:, 1])

In [ ]:
import matplotlib.pyplot as plt

def plot1d(x, x2=None, x3=None, ylim=(-1, 1), save_file=""):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(6, 3))
    steps = np.arange(x.shape[0])
    plt.plot(steps, x)
    if x2 is not None:
        plt.plot(steps, x2)
    if x3 is not None:
        plt.plot(steps, x3)
    plt.xlim(0, x.shape[0])
    # plt.ylim(ylim)
    plt.tight_layout()
    if save_file:
        plt.savefig(save_file, "")
    else:
        plt.show()
    return

In [ ]:
pos = 90
plot1d(mts.X[pos])

In [ ]:
plot1d(rec[pos])